In [103]:
import pandas as pd
import os
from datetime import datetime
from datetime import date


In [104]:

allFiles = os.listdir('scrapedData')
csvFileNames = []
for fileName in allFiles:
    if "csvOut" in fileName:
        csvFileNames.append(fileName)
        

combined_csv = pd.concat( [ pd.read_csv("scrapedData\\" + f) for f in csvFileNames ] )        


In [105]:
pd.options.display.max_columns = None
#combined_csv = pd.read_csv("scrapedData\\" + csvFileNames[0])
numRows = len(combined_csv.index)
print(numRows)
columnCountCurrent = (len(combined_csv.index))
notAvailableColumn = ["#NA"] * numRows
newStringColumn = ["new"] * numRows


combined_csv['thumbnail_link'] = notAvailableColumn
combined_csv['comments_disabled'] = notAvailableColumn
combined_csv['ratings_disabled'] = notAvailableColumn
combined_csv['video_error_or_removed'] = notAvailableColumn

# Creating publishedDateCorrectFormat column
videoPublishTimeColumn = (combined_csv['videoPublishTime'])
videoPublishTimeColumnSplit1 = [i.split('T', 1)[0] for i in videoPublishTimeColumn]
videoPublishTimeStandard = [datetime.strptime(i, '%Y-%m-%d') for i in videoPublishTimeColumnSplit1]    # THis is in standard datetime object format
publishedDateCorrectFormat = [x.strftime("%d-%m-%y") for x in videoPublishTimeStandard]
combined_csv['publishedDateCorrectFormat'] = publishedDateCorrectFormat




# Creating trendingDateCorrectFormat
timeFetchedColumn = combined_csv['timeFetched']
timeFetchedColumnSplit1 = [i.split(' ', 1)[0] for i in timeFetchedColumn]
timeFetchedStandard = [datetime.strptime(i, '%Y-%m-%d') for i in timeFetchedColumnSplit1]
trendingDateCorrectFormat = [x.strftime("%d-%m-%y") for x in timeFetchedStandard]
combined_csv['trendingDateCorrectFormat'] = trendingDateCorrectFormat


# Calculating day difference
dayDifference = [(timeFetchedStandard[i] - videoPublishTimeStandard[i]).days for i in range(numRows)]
combined_csv['dayDifference'] = dayDifference

# newOrOldData column
combined_csv['newOrOldData'] = newStringColumn


combined_csv.head()

10800


,regionTrending,trendingRank,timeFetched,videoId,videoTitle,videoCategoryId,videoPublishTime,videoDuration,videoTags,videoViews,videoLikes,videoDislikes,videoCommentCount,videoDescription,vieoLicenced,channelName,channelId,channelDescription,channelPublishedAt,channelViewCount,channelSubsCount,channelVideoCount,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,publishedDateCorrectFormat,trendingDateCorrectFormat,dayDifference,newOrOldData
0,US,1,2020-07-01 15:09:45.453481,h0U2QUGKbSE,Kanye West – Wash Us In The Blood feat. Travis...,22,2020-06-30T14:00:11Z,PT3M42S,"['kanye', 'kanye west', 'ye', 'yeezus', 'yeezy...",4214043.0,269209.0,13107.0,22665.0,Stream/Download “Wash Us In The Blood” ft. Tra...,False,Kanye West,UCs6eXM7s8Vl5WcECcRHc2qQ,NaN,2006-01-10T22:52:29Z,40986002,6200000,9,#NA,#NA,#NA,#NA,30-06-20,01-07-20,1,new
1,US,2,2020-07-01 15:09:45.453481,LiB65FQnm6w,Cash vs Flight 1v1 Basketball! Shave Beard or ...,24,2020-06-30T17:53:01Z,PT25M42S,"['cash vs flight 1v1 basketball', 'flight reac...",1497970.0,103096.0,1083.0,10384.0,Get Your Energy Like Me Using GG! \nI receive ...,True,CashNasty,UCvyTdLw8SkVmUcHYXSDEGwA,I make videos. I make you laugh. I be happy.,2013-06-22T01:48:44Z,758385956,3630000,2132,#NA,#NA,#NA,#NA,30-06-20,01-07-20,1,new
2,US,3,2020-07-01 15:09:45.453481,T8pi91qWnRw,Moneybagg Yo – Said Sum (Official Music Video),10,2020-06-30T16:59:58Z,PT2M59S,NaN,689435.0,51327.0,916.0,1915.0,Moneybagg Yo's new track 'Said Sum' out now: h...,False,MoneyBagg Yo,UCrdPrDuDCbG8xayk5QkRLQA,NaN,2016-10-06T03:39:25Z,344603686,1300000,84,#NA,#NA,#NA,#NA,30-06-20,01-07-20,1,new
3,US,4,2020-07-01 15:09:45.453481,rAxNSpO78fE,I Surprised Brent Rivera With A Custom iPad Pr...,24,2020-06-30T16:59:43Z,PT13M59S,NaN,2863644.0,212914.0,3235.0,64229.0,"This was Insane, I Can't Believe We Did This F...",True,ZHC,UClQubH2NeMmGLTLgNdLBwXg,Thanks for subscribing! Its my mission to make...,2013-08-07T03:22:54Z,914474888,13700000,272,#NA,#NA,#NA,#NA,30-06-20,01-07-20,1,new
4,US,5,2020-07-01 15:09:45.453481,mrF-KjnDTxc,"Genoa 1-3 Juventus | Dybala, CR7 & Douglas Cos...",17,2020-06-30T22:30:01Z,PT4M14S,"['Dybala', 'CR7', 'Douglas Costa', 'Genoa', 'D...",4259625.0,106818.0,1644.0,4661.0,Juventus re-open their four point lead at the ...,True,Serie A,UCBJeMCIeLQos7wacox4hmLQ,Welcome to the Official Serie A channel. Over ...,2012-10-30T13:54:30Z,1406914395,4660000,18934,#NA,#NA,#NA,#NA,30-06-20,01-07-20,1,new


In [106]:
combined_csv.to_csv( "newDataOnly_csv_newFormat.csv", index=False )

In [107]:
oldData = pd.read_excel (r'finalOldData.xlsx')

In [111]:
print(oldData.columns)
print(combined_csv.columns)
print(combined_csv.columns==oldData.columns)
combinedOldNew_csv = pd.concat( [oldData, combined_csv] )        
combinedOldNew_csv.to_csv( "oldAndNewData.csv", index=False )

Index(['regionTrending', 'trendingRank', 'timeFetched', 'videoId',
       'videoTitle', 'videoCategoryId', 'videoPublishTime', 'videoDuration',
       'videoTags', 'videoViews', 'videoLikes', 'videoDislikes',
       'videoCommentCount', 'videoDescription', 'videoLicenced',
       'channelTitle', 'channelId', 'channelDescription', 'channelPublishedAt',
       'channelViewCount', 'channelSubsCount', 'channelVideoCount',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'publishDateCorrectFormat',
       'trendingDateCorrectFormat', 'dayDifference', 'newOrOldData'],
      dtype='object')
Index(['regionTrending', 'trendingRank', 'timeFetched', 'videoId',
       'videoTitle', 'videoCategoryId', 'videoPublishTime', 'videoDuration',
       'videoTags', 'videoViews', 'videoLikes', 'videoDislikes',
       'videoCommentCount', 'videoDescription', 'vieoLicenced', 'channelName',
       'channelId', 'channelDescription', 'channelPublishedAt',
       